In [4]:
import requests

In [5]:
# 민원정책 질의응답조회서비스
url = 'http://apis.data.go.kr/1140100/CivilPolicyQnaService/PolicyQnaList'



In [2]:
import os
e_key = os.getenv('SINNUNGO_ENCODING_KEY', 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789')
d_key = os.getenv('SINNUNGO_DECODING_KEY', 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789')

In [ ]:
# 민원정책 질의응답조회서비스
url = 'http://apis.data.go.kr/1140100/CivilPolicyQnaService/PolicyQnaList'

params ={
    'serviceKey' : d_key, 
    'firstIndex' : '1', 
    'recordCountPerPage' : '30000', 
    # 'type' : '1', 
    # 'keyword' : '궁금', 
    # 'searchType' : '1', 
    # 'regFrom' : '20240506', 
    # 'regTo' : '20240508' 
}

response = requests.get(url, params=params)
r = response.json()
faq_list = r['resultList']
len(faq_list)


20072

In [27]:
print(len(faq_list))

from datetime import datetime

now = datetime.now()
print(now.strftime("%Y-%m-%d %H:%M:%S"))



20072
2025-05-07 17:18:15


In [24]:
l = data['resultList']
len(l)

20072

# 민원정책 질의응답 항목 조회

In [59]:
url = 'http://apis.data.go.kr/1140100/CivilPolicyQnaService/PolicyQnaItem'

params ={
    'serviceKey' : d_key,
    'faqNo': 6859437,
    'dutySctnNm': 'tqapttn'
}

response = requests.get(url, params=params)
response.json()

{'resultCode': 'S00',
 'resultMessage': '1(건) 조회되었습니다.',
 'resultData': {'faqNo': '6859437',
  'dutySctnNm': 'tqapttn',
  'qnaTitl': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?',
  'qstnCntnCl': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?',
  'ansCntnCl': '&lt;span style=&quot;font-family:KoPub돋움체 Light&quot;&gt;해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ&lt;/span&gt;&lt;a&gt;&lt;span style=&quot;font-family:KoPub돋움체 Light&quot;&gt;특별자치도&lt;/span&gt;(&lt;span style=&quot;font-family:KoPub돋움체 Light&quot;&gt;관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다&lt;/span&gt;)&lt;/a&gt; &lt;span style=&quot;font-family:KoPub돋움체 Light&quot;&gt;또는 시ㆍ군의 행정구역을 말합니다&lt;/span&gt;. &lt;span style=&quot;font-family:KoPub돋움체 Light&quot;&gt;따라서&lt;/span&gt;, &lt;span style=&quot;font-family:KoPub돋움체 Light&quot;&gt;경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다&lt;/span&gt;.&lt;br /&gt;\n&lt;span style=&quot;font-family:KoPub돋움체 Light&quot;&gt;참&lt;/span&gt;&lt;span style=&quot;font-family:KoPub돋움체 Light&quot;&gt;고로&lt;/span&gt;, &lt;span style=&quot;fo

In [74]:
url = 'http://apis.data.go.kr/1140100/CivilPolicyQnaService/PolicyQnaItem'

params ={
    'serviceKey' : d_key,
    'faqNo': 1192000,
    'dutySctnNm': 'tqaplcy'
}

response = requests.get(url, params=params)
response.json()

{'resultCode': 'E03',
 'resultMessage': '조회된 데이터가 없습니다.',
 'resultDebug': '2025-05-01 19:01:07, eapi22'}

# DB 에 저장

In [19]:
import sqlite3
import json

In [20]:
type(data)

dict

In [25]:
data['resultList'][1]

{'dutySctnNm': 'tqapttn',
 'ancCode': '7480000',
 'ancName': '울산광역시교육청',
 'regDate': '20250506092434',
 'title': '울산마을교육공동체 시설 이용 관련 안내',
 'faqNo': '6882492'}

In [26]:
len(data['resultList'])

20072

In [27]:
# SQLite DB 연결 (없으면 생성됨)
conn = sqlite3.connect('faq20072.db')
cursor = conn.cursor()

In [28]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS faq (
    faqNo TEXT PRIMARY KEY,
    dutySctnNm TEXT,
    ancCode TEXT,
    ancName TEXT,
    regDate TEXT,
    title TEXT
)
''')

In [29]:
# 데이터 삽입
for item in data['resultList']:
    cursor.execute('''
        INSERT OR REPLACE INTO faq (faqNo, dutySctnNm, ancCode, ancName, regDate, title)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        item['faqNo'],
        item['dutySctnNm'],
        item['ancCode'],
        item['ancName'],
        item['regDate'],
        item['title']
    ))

In [30]:
# 저장 및 종료
conn.commit()
conn.close()

In [ ]:
import sqlite3
import requests
import time  # 요청 간 텀 주기 위해

# 인증키 및 API 기본 정보
API_URL = 'http://apis.data.go.kr/1140100/CivilPolicyQnaService/PolicyQnaItem'
API_KEY = d_key  # 반드시 디코딩된 형태로 넣어야 함

# SQLite 연결
conn = sqlite3.connect('/home/tom/code/sinmun-go/note/faq20072.db')
cursor = conn.cursor()

# 상세 정보 저장용 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS faq_detail (
    faqNo TEXT PRIMARY KEY,
    qnaTitl TEXT,
    qstnCntnCl TEXT,
    ansCntnCl TEXT,
    ancName TEXT,
    deptName TEXT,
    regDate TEXT,
    ancCode TEXT,
    deptCode TEXT
)
''')

# faq 테이블에서 필요한 항목 조회
cursor.execute('SELECT faqNo, dutySctnNm FROM faq ORDER BY faqNo')
faq_items = cursor.fetchall()

# 루프 돌며 상세 정보 가져와 DB 저장
for faqNo, dutySctnNm in faq_items:
    params = {
        'serviceKey': API_KEY,
        'faqNo': faqNo,
        'dutySctnNm': dutySctnNm
    }

    try:
        response = requests.get(API_URL, params=params)
        data = response.json()
        if data['resultCode'] != 'S00':
            print(f"[{faqNo}] API 오류: {data['resultMessage']}")
            continue

        item = data['resultData']
        cursor.execute('''
            INSERT OR REPLACE INTO faq_detail (
                faqNo, qnaTitl, qstnCntnCl, ansCntnCl,
                ancName, deptName, regDate, ancCode, deptCode
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            item.get('faqNo'),
            item.get('qnaTitl'),
            item.get('qstnCntnCl'),
            item.get('ansCntnCl'),
            item.get('ancName'),
            item.get('deptName'),
            item.get('regDate'),
            item.get('ancCode'),
            item.get('deptCode')
        ))
        # print(f"[{faqNo}] 저장 완료")
        # time.sleep(0.3)  # 과도한 요청 방지

    except Exception as e:
        print(f"[{faqNo}] 처리 중 예외 발생: {e}")

# 저장 및 종료
conn.commit()
conn.close()


[6882504] 저장 완료
[6882492] 저장 완료
[6882491] 저장 완료
[6882489] 저장 완료
[6882490] 저장 완료
[6882488] 저장 완료
[6882376] 저장 완료
[6882388] 저장 완료
[6880877] 저장 완료
[6882127] 저장 완료
[6882439] 저장 완료
[6882480] 저장 완료
[6882499] 저장 완료
[6882509] 저장 완료
[6882193] 저장 완료
[6882259] 저장 완료
[6882487] 저장 완료
[6882397] 저장 완료
[6882423] 저장 완료
[6882422] 저장 완료
[6882436] 저장 완료
[6882418] 저장 완료
[6882416] 저장 완료
[6882421] 저장 완료
[6882121] 저장 완료
[6882117] 저장 완료
[6882114] 저장 완료
[6882083] 저장 완료
[6882082] 저장 완료
[6882076] 저장 완료
[6882068] 저장 완료
[6882051] 저장 완료
[6882025] 저장 완료
[6882016] 저장 완료
[6882015] 저장 완료
[6882010] 저장 완료
[6882008] 저장 완료
[6882007] 저장 완료
[6882005] 저장 완료
[6881990] 저장 완료
[6882441] 저장 완료
[6882000] 저장 완료
[6881989] 저장 완료
[6881975] 저장 완료
[6881970] 저장 완료
[6881940] 저장 완료
[6881941] 저장 완료
[6881944] 저장 완료
[6881943] 저장 완료
[6881924] 저장 완료
[6881847] 저장 완료
[6881792] 저장 완료
[6881484] 저장 완료
[6882089] 저장 완료
[6882085] 저장 완료
[6882098] 저장 완료
[6882215] 저장 완료
[6882292] 저장 완료
[6882192] 저장 완료
[6882253] 저장 완료
[6881824] 저장 완료
[6882508] 저장 완료
[6882507

KeyboardInterrupt: 

In [4]:
conn.commit()
conn.close()